In [1]:
from graphframes import *

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc


 

In [6]:
import os
from pyspark.sql import SparkSession
os.environ['PYSPARK_PYTHON'] = "/usr/bin/python3"
spark = SparkSession.builder.appName("graph processing")\
.config("spark.jars.packages","graphframes:graphframes:0.8.2-spark3.2-s_2.12") \
.master("local[*]").getOrCreate()

25/03/24 03:07:46 WARN Utils: Your hostname, bigdata2024 resolves to a loopback address: 127.0.1.1; using 10.3.132.216 instead (on interface ens3)
25/03/24 03:07:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hadoop/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-aac90f49-1445-41d4-8f35-565612074a08;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (77ms)
:: resolution report :: resolve 2396ms :: artifacts dl 84ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number

In [7]:
v= spark.createDataFrame([('1', 'Carter', 'Derrick', 50), 
                                  ('2', 'May', 'Derrick', 26),
                                 ('3', 'Mills', 'Jeff', 80),
                                  ('4', 'Hood', 'Robert', 65),
                                  ('5', 'Banks', 'Mike', 93),
                                 ('98', 'Berg', 'Tim', 28),
                                 ('99', 'Page', 'Allan', 16)],
                                 ['id', 'name', 'firstname', 'age'])#create edges dataframe

In [8]:
e = spark.createDataFrame([('1', '2', 'friend'), 
                               ('2', '1', 'friend'),
                              ('3', '1', 'friend'),
                              ('1', '3', 'friend'),
                               ('2', '3', 'follows'),
                               ('3', '4', 'friend'),
                               ('4', '3', 'friend'),
                               ('5', '3', 'friend'),
                               ('3', '5', 'friend'),
                               ('4', '5', 'follows'),
                              ('98', '99', 'friend'),
                              ('99', '98', 'friend')],
                              ['src', 'dst', 'type'])

In [9]:
g = GraphFrame(v, e)

/home/hadoop/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [ ]:
g.inDegrees.show()

/home/hadoop/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [ ]:
g.edges.filter("type = 'follows'").count()

In [ ]:
 g.vertices.show()

In [ ]:
g.edges.show()

In [9]:
inDegreeDF=g.inDegrees
outDegreeDF=g.outDegrees
degreeDF=g.degrees

inDegreeDF.sort(['inDegree'],ascending=[0]).show() # Sort and show

+---+--------+
| id|inDegree|
+---+--------+
|  3|       4|
|  5|       2|
|  1|       2|
|  4|       1|
| 98|       1|
|  2|       1|
| 99|       1|
+---+--------+



In [10]:
outDegreeDF.sort(['outDegree'],ascending=[0]).show()

+---+---------+
| id|outDegree|
+---+---------+
|  3|        3|
|  1|        2|
|  4|        2|
|  2|        2|
|  5|        1|
| 98|        1|
| 99|        1|
+---+---------+



In [11]:
results = g.pageRank(resetProbability=0.01, maxIter=20)

In [12]:
results3 = g.pageRank(resetProbability=0.15, maxIter=10, sourceId="1")

In [13]:
results.vertices.select("id", "pagerank").show()

+---+-------------------+
| id|           pagerank|
+---+-------------------+
|  1|  0.890802837490737|
|  3| 1.9926565142372492|
|  2|0.45093462280858443|
|  4|   0.66757817627641|
| 98| 1.0000000000000004|
|  5| 0.9980278491870183|
| 99| 1.0000000000000004|
+---+-------------------+



In [14]:
results3.vertices.select("id", "pagerank").show()

+---+-------------------+
| id|           pagerank|
+---+-------------------+
|  1|0.30079283470815427|
|  3|0.33891806394017526|
|  2| 0.1272219398157718|
|  4|0.09616868544059574|
| 98|                0.0|
|  5|0.13689847609530284|
| 99|                0.0|
+---+-------------------+



In [15]:
results = g.pageRank(resetProbability=0.15, tol=0.01)

In [16]:
results.vertices.select("id", "pagerank").show()

+---+------------------+
| id|          pagerank|
+---+------------------+
|  1|0.9055074972891308|
|  3| 1.853919642738813|
|  2|0.5377967999474921|
|  4|0.6873519241384106|
| 98|1.0225331112091938|
|  5|0.9703579134677663|
| 99|1.0225331112091938|
+---+------------------+



In [17]:
from graphframes.examples import Graphs

In [18]:
g = Graphs(sqlContext).friends()  # Get example graph

In [19]:
g.edges.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  c|      follow|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  e|  d|      friend|
|  d|  a|      friend|
+---+---+------------+



In [20]:
g.vertices.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 30|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 36|
+---+-------+---+



In [21]:
motifs = g.find("(a)-[e]->(b); (b)-[e2]->(a)")

In [22]:
motifs.show()

+----------------+--------------+----------------+--------------+
|               a|             e|               b|            e2|
+----------------+--------------+----------------+--------------+
|[c, Charlie, 30]|[c, b, follow]|    [b, Bob, 36]|[b, c, follow]|
|    [b, Bob, 36]|[b, c, follow]|[c, Charlie, 30]|[c, b, follow]|
+----------------+--------------+----------------+--------------+



In [23]:
motifs.filter("b.age > 30").show()

+----------------+--------------+------------+--------------+
|               a|             e|           b|            e2|
+----------------+--------------+------------+--------------+
|[c, Charlie, 30]|[c, b, follow]|[b, Bob, 36]|[b, c, follow]|
+----------------+--------------+------------+--------------+



In [24]:
mutualFriends = g.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(b); (b)-[]->(a)").dropDuplicates()

In [25]:
mutualFriends.filter('a.id == 2 and c.id == 3').show()

+---+---+---+
|  a|  b|  c|
+---+---+---+
+---+---+---+

